In [1]:
from utils.molecule_tools import get_molecule
from utils.cs_vqe_tools_original import greedy_dfs
import json

/home/tim/anaconda3/lib/python3.7/site-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')
/home/tim/anaconda3/lib/python3.7/site-packages/qiskit/aqua/components/optimizers/optimizer.py:50: DeprecationWarning: The package qiskit.aqua.components.optimizers is deprecated. It was moved/refactored to qiskit.algorithms.optimizers (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  'qiskit.algorithms.optimizers', 'qiskit-terra')


In [2]:
file = 'molecule_data'
with open('data/'+file+'.json', 'r') as json_file:
    molecule_data = json.load(json_file)

speciesname_list = list(molecule_data.keys())
speciesname_list

['H2_3-21G_SINGLET',
 'H6_STO-3G_SINGLET',
 'H2_6-31G_SINGLET',
 'H2_6-311G_SINGLET',
 'H3+_STO-3G_SINGLET',
 'H3+_3-21G_SINGLET',
 'H3+_STO-3G_SINGLET_alt',
 'HeH+_3-21G_SINGLET',
 'HeH+_3-21G_SINGLET_alt',
 'HeH+_6-311G_SINGLET',
 'H2O_STO-3G_SINGLET',
 'H2O_STO-3G_SINGLET_alt',
 'BeH+_STO-3G_SINGLET',
 'LiH_STO-3G_SINGLET',
 'CH+_STO-3G_SINGLET',
 'HF_STO-3G_SINGLET',
 'B+_STO-3G_SINGLET',
 'N+_STO-3G_SINGLET',
 'OH+_STO-3G_SINGLET',
 'O_STO-3G_SINGLET',
 'CH2_STO-3G_SINGLET',
 'BeH2_STO-3G_SINGLET',
 'Be_STO-3G_SINGLET',
 'C_STO-3G_SINGLET',
 'NH_STO-3G_SINGLET',
 'Ne_STO-3G_SINGLET',
 'F+_STO-3G_SINGLET',
 'Li+_STO-3G_SINGLET',
 'BH_STO-3G_SINGLET',
 'NeH+_STO-3G_SINGLET',
 'NH2+_STO-3G_SINGLET',
 'BH2+_STO-3G_SINGLET']

In [5]:
speciesname_list[-6:]

['F+_STO-3G_SINGLET',
 'Li+_STO-3G_SINGLET',
 'BH_STO-3G_SINGLET',
 'NeH+_STO-3G_SINGLET',
 'NH2+_STO-3G_SINGLET',
 'BH2+_STO-3G_SINGLET']

In [7]:
file = 'model_data'
with open('data/model_data.json', 'r') as json_file:
    model_data = json.load(json_file)
model_data.keys()

dict_keys(['H2_3-21G_SINGLET', 'H6_STO-3G_SINGLET', 'H2_6-31G_SINGLET', 'H2_6-311G_SINGLET', 'H3+_STO-3G_SINGLET', 'H3+_3-21G_SINGLET', 'H3+_STO-3G_SINGLET_alt', 'HeH+_3-21G_SINGLET', 'HeH+_3-21G_SINGLET_alt', 'HeH+_6-311G_SINGLET', 'H2O_STO-3G_SINGLET', 'H2O_STO-3G_SINGLET_alt', 'BeH+_STO-3G_SINGLET', 'LiH_STO-3G_SINGLET', 'CH+_STO-3G_SINGLET', 'HF_STO-3G_SINGLET', 'B+_STO-3G_SINGLET', 'N+_STO-3G_SINGLET', 'OH+_STO-3G_SINGLET', 'CH2_STO-3G_SINGLET', 'BeH2_STO-3G_SINGLET', 'Be_STO-3G_SINGLET', 'C_STO-3G_SINGLET', 'NH_STO-3G_SINGLET'])

In [9]:
#model_data = {}
for speciesname in speciesname_list[-6:]:
    print(speciesname)
    molecule = get_molecule(speciesname, taper=True)
    num_qubits  = molecule['num_qubits']
    ham         = molecule['hamiltonian']
    uccsd       = molecule['uccsdansatz']
    hf_config   = molecule['hf_config']
    num_tapered = molecule['num_tapered']
    true_gs_nrg = molecule['true_gs_nrg']
    true_gs_vec = molecule['true_gs_vec']
    
    for p in ham.keys():
        ham[p] = ham[p].real    
    for p in uccsd.keys():
        uccsd[p] = uccsd[p].imag
    
    terms_noncon = greedy_dfs(ham, 10, criterion='size')[-1]
    
    model_data[speciesname] = {"ham":ham,
                               "uccsd":uccsd,
                               "num_qubits":num_qubits,
                               "hf_config":hf_config,
                               "terms_noncon":terms_noncon,
                               "num_tapered":num_tapered,
                               "true_gs_nrg":true_gs_nrg}
                               #"true_gs_vec":true_gs_vec}
    print('Molecule constructed.')

F+_STO-3G_SINGLET
*** sector saved, will check tapered ground state energy matches target problem ***
Attempting to taper 10 --> 5 qubits
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Energies match in sector (1, -1, 1, 1, 1), tapering successful!

Molecule constructed.
Li+_STO-3G_SINGLET
*** sector saved, will check tapered ground state energy matches target problem ***
Attempting to taper 10 --> 5 qubits
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Energies match in sector (1, 1, 1, -1, 1), tapering successful!

Molecule constructed.
BH_STO-3G_SINGLET
*** sector saved, will check tapered ground state energy matches target problem ***
Attempting to taper 12 --> 8 qubits
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Energies match in sector (1, 1, -1, -1), tapering successful!

Molecule constructed.
NeH+_STO-3G_SINGLET
*** sector saved, will check tapered ground state energy matches target problem ***
Attempting to taper 12 --> 8 qubits
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Energies match in sector (1, 1, -1, 1)

In [10]:
data={}
for k in model_data.keys():
    data[str(k)] = model_data[k]
data

{'H2_3-21G_SINGLET': {'ham': {'IIIII': 2.5709195146451136,
   'ZIIII': -1.1164555268534126,
   'IZIII': -1.1164555268534126,
   'ZZIII': 0.28873425268242214,
   'IIZII': -0.7911196771178006,
   'ZIZII': 0.019110391453146282,
   'IZZII': 0.0323898209816923,
   'ZZZII': -3.469446951953614e-18,
   'IIIZI': -0.7911196771178004,
   'ZIIZI': 0.03238982098169225,
   'IZIZI': 0.019110391453146303,
   'ZZIZI': -3.469446951953614e-18,
   'IIZZI': 0.28908129604041466,
   'ZIZZI': 8.673617379884035e-18,
   'IZZZI': 1.1449174941446927e-16,
   'ZZZZI': -3.469446951953614e-18,
   'IIIIZ': -0.4399036642033591,
   'ZIIIZ': -0.025103665863977165,
   'IZIIZ': 0.23398912301362987,
   'ZZIIZ': -0.43990366420335913,
   'IIZIZ': -0.03979331187825033,
   'ZIZIZ': 0.2989751415259368,
   'IZZIZ': -1.734723475976807e-18,
   'ZZZIZ': -0.06354270641645747,
   'IIIZZ': -0.06354270641645746,
   'ZIIZZ': 0.2989751415259367,
   'IZIZZ': 5.204170427930421e-18,
   'ZZIZZ': -0.03979331187825033,
   'IIZZZ': 8.67361737988

In [9]:
model_data

{}

In [11]:
data={}
for k in model_data.keys():
    data[str(k)] = model_data[k]
     
with open("data/model_data.json", "w") as outfile: 
    json.dump(data, outfile)

In [21]:
len(speciesname_list)

24